In [1]:
from fastai.vision import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callbacks import ActivationStats, SaveModelCallback

In [2]:
from gepcore.utils import cell_graph, convolution
from gepcore.entity import Gene, Chromosome
from gepcore.symbol import PrimitiveSet
from gepnet.model_v2 import get_gepnet, arch_config
from gepnet.utils import count_parameters
torch.backends.cudnn.benchmark = True

In [3]:
from pygraphviz import AGraph
import glob

In [5]:
# get chromosme from file
graph = [AGraph(g) for g in glob.glob('nb_graphs/rs_01/*.dot')]
graph, comp_graph = cell_graph.generate_comp_graph(graph)

In [ ]:
# generate new chromosome
# define primitive set
pset = PrimitiveSet('cnn')

# add cellular encoding program symbols
pset.add_program_symbol(cell_graph.end)
pset.add_program_symbol(cell_graph.seq)
pset.add_program_symbol(cell_graph.cpo)
pset.add_program_symbol(cell_graph.cpi)

# add convolutional operations symbols
conv_symbol = convolution.get_symbol()
pset.add_cell_symbol(conv_symbol.conv1x1)
pset.add_cell_symbol(conv_symbol.conv3x3)
pset.add_cell_symbol(conv_symbol.dwconv3x3)
pset.add_cell_symbol(conv_symbol.conv1x3)
pset.add_cell_symbol(conv_symbol.conv3x1)
pset.add_cell_symbol(conv_symbol.maxpool3x3)

def gene_gen():
    return Gene(pset, 3)

ch = Chromosome(gene_gen, 2)
graph, comp_graph = cell_graph.generate_comp_graph(ch)

cell_graph.save_graph(graph, 'nb_graphs/rs_01')
cell_graph.draw_graph(graph, 'nb_graphs/rs_01')

In [6]:
import random
seed = 221
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# enable torch backends
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
#torch.backends.deterministic = True

In [7]:
conf = arch_config(comp_graph=comp_graph,
                   depth_coeff=1.0,
                   width_coeff=1.0,
                   channels=64,
                   repeat_list=[3, 4, 2],
                   classes=10)

net = get_gepnet(conf)
count_parameters(net)

3.429962

In [9]:
path = untar_data(URLs.CIFAR)
tfms = get_transforms() #[cutout(n_holes=(1,4), length=(10, 160), p=1.)](xtra_tfms=rand_resize_crop(32)

bs = 128
data = (ImageList.from_folder(path/'train')
        .split_by_rand_pct(valid_pct=0.1, seed=seed) 
        .label_from_folder()
        .transform(tfms, size=32)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize(cifar_stats))

In [10]:
learn = Learner(data, net, metrics=[error_rate, accuracy], 
                model_dir='/home/cliff/ResearchProjects/models/random_search/').mixup()
#learn.to_fp16()

In [11]:
cb = SaveModelCallback(learn, every='improvement', monitor='accuracy', name='exp1_model')
learn.fit_one_cycle(300, 1e-3, wd=0.0004, callbacks=[cb]) 

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.693432,0.222332,0.057400,0.942600,01:30
1,0.677628,0.236914,0.058600,0.941400,01:29
2,0.690224,0.221786,0.058600,0.941400,01:30
3,0.683763,0.218813,0.058400,0.941600,01:30
4,0.677463,0.211846,0.057200,0.942800,01:31
5,0.687426,0.212913,0.057400,0.942600,01:31
6,0.679043,0.210160,0.058200,0.941800,01:31
7,0.675638,0.233891,0.059000,0.941000,01:31
8,0.669516,0.208070,0.056200,0.943800,01:31
9,0.672670,0.208317,0.055800,0.944200,01:31


Better model found at epoch 0 with accuracy value: 0.9426000118255615.
Better model found at epoch 4 with accuracy value: 0.942799985408783.
Better model found at epoch 8 with accuracy value: 0.9437999725341797.
Better model found at epoch 9 with accuracy value: 0.9441999793052673.
Better model found at epoch 14 with accuracy value: 0.9448000192642212.
Better model found at epoch 23 with accuracy value: 0.9452000260353088.
Better model found at epoch 33 with accuracy value: 0.9466000199317932.
Better model found at epoch 37 with accuracy value: 0.948199987411499.
Better model found at epoch 53 with accuracy value: 0.9490000009536743.
Better model found at epoch 67 with accuracy value: 0.949999988079071.
Better model found at epoch 85 with accuracy value: 0.9503999948501587.
Better model found at epoch 122 with accuracy value: 0.9513999819755554.
Better model found at epoch 123 with accuracy value: 0.9516000151634216.


In [ ]:
## Testing

In [ ]:
graph = [AGraph(g) for g in glob.glob('comp_graphs/experiment_1/best/indv_0/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)

In [ ]:
conf = arch_config(comp_graph=comp_graph,
                   depth_coeff=1.0,
                   width_coeff=1.0,
                   channels=64,
                   repeat_list=[5, 5, 5],
                   classes=10)

net = get_gepnet(conf)
count_parameters(net)

In [ ]:
tfms = get_transforms(do_flip=False)
path = untar_data(URLs.CIFAR)
bs = 64

data = (ImageList.from_folder(path)
        .split_by_folder(train='train', valid='test')
        .label_from_folder()
        .transform(tfms, size=32)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize(cifar_stats))

In [ ]:
model = Learner(data, net, metrics=[accuracy, error_rate]).load('experiment_1_model_0')
#model

In [ ]:
model.validate(data.valid_dl)

In [ ]:
interp = ClassificationInterpretation.from_learner(model)
losses,idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(10,5), heatmap=False)

In [ ]:
interp.plot_confusion_matrix(figsize=(8,6), dpi=65)

In [ ]:
#preds,y,losses = model.get_preds(ds_type=DatasetType.Valid, with_loss=True)
#preds[:5]

In [ ]:
import pickle

In [ ]:
with open('comp_graphs/experiment_1/best/stats.pkl', 'rb') as f:
    stats = pickle.load(f)

In [ ]:
print(stats)